## Modifying Generated Image using Amazon Titan Image Generator

In [ ]:
import boto3
import json
import base64
import io
import os
from PIL import Image
import uuid

# Initialize Amazon Bedrock client
bedrock_client = boto3.client("bedrock-runtime", region_name="us-east-1")

# Define the model ID
model_id = "amazon.titan-image-generator-v2:0"

# Step 1: Generate Initial Image
# Prepare the input prompt
initial_prompt = "Create me an image of a cricket player with bat in one hand which is held on the shoulder and helmet in another hand and the player should have wear blue jersey with team logo of mumbai indians"

# Create the request payload
payload = {
    "taskType": "TEXT_IMAGE",
    "textToImageParams": {"text": initial_prompt},
    "imageGenerationConfig": {
        "numberOfImages": 1,
        "height": 1024,
        "width": 1024,
        "cfgScale": 8.0,
        "seed": 0
    }
}

# Invoke the Amazon Titan Image Generator v2 model
response = bedrock_client.invoke_model(
    modelId=model_id,
    body=json.dumps(payload),
    accept="application/json",
    contentType="application/json"
)

# Process the response
result = json.loads(response["body"].read())

base64_image = result.get("images")[0]
base64_bytes = base64_image.encode('ascii')
image_bytes = base64.b64decode(base64_image)

initial_image = Image.open(io.BytesIO(image_bytes))
initial_image.show()

unique_id = uuid.uuid4()
os.mkdir(f"generated-and-modified-images/{unique_id}")

initial_image_path = f"generated-and-modified-images/{unique_id}/generated_image.png"
initial_image.save(initial_image_path)
print(f"Generated image saved at: {initial_image_path}")

# Step 2: Modifying the Generated Image (with an updated prompt)
modified_prompt = "Change the player's complete jersey color to yellow and team logo name to chennai super kings in the previously generated image."

with open(f"D:\Data-AI Blogs\Blog-2\generated-and-modified-images\{unique_id}\generated_image.png","rb") as image_file:
    base64_image = base64.b64encode(image_file.read()).decode("utf-8")

# Update Payload for Image Updation
payload = {
     "taskType": "IMAGE_VARIATION",
     "imageVariationParams": {
         "text": modified_prompt,
         "images": [base64_image],
         "similarityStrength": 1.0,  # Range: 0.2 to 1.0
     },
     "imageGenerationConfig": {
         "quality": "premium",
         "numberOfImages": 1,
         "height": 1024,
         "width": 1024,
         "cfgScale": 8.0
     }
}

response = bedrock_client.invoke_model(
    modelId=model_id,
    body=json.dumps(payload),
    accept="application/json",
    contentType="application/json"
)

result = json.loads(response["body"].read())

base64_modified_image = result.get("images")[0]
base64_bytes = base64_image.encode('ascii')
modified_image_bytes = base64.b64decode(base64_modified_image)

# Save and display the modified image
modified_image = Image.open(io.BytesIO(modified_image_bytes))
modified_image.show()

modified_image_path = f"generated-and-modified-images/{unique_id}/modified_image.png"
modified_image.save(modified_image_path)
print(f"Modified image saved at: {modified_image_path}")

Generated image saved at: generated-and-modified-images/6ba0e04c-def5-4efb-a710-83b35d2231f2/generated_image.png
Modified image saved at: generated-and-modified-images/6ba0e04c-def5-4efb-a710-83b35d2231f2/modified_image.png
